In [1]:
""" 
This notebook illustrates how to retrieve player statistics and convert them into player profiles 
(offensive, defensive, neutral). It also shows how to retrieve game info, such as players list etc.
"""


' \nThis notebook illustrates how to retrieve player statistics and convert them into player profiles \n(offensive, defensive, neutral). It also shows how to retrieve game info, such as players list etc.\n'

In [2]:
from ReinforcementLearning.NHL.playerstats.nhl_player_stats import *
from ReinforcementLearning.NHL.playbyplay.game import *

In [3]:
# Pointers to the data
#repoCode    =   '/home/younesz/Documents/Code/Python'
repoCode    =   '/Users/younes_zerouali/Documents/Stradigi/Code/NHL_stats_SL'
#db_root     =   '/home/younesz/Documents/Hockey'        #This is the location of the Hockey database
db_root     =   '/Users/younes_zerouali/Documents/Stradigi/Databases/Hockey'
repoPbP     =   path.join(db_root, 'PlayByPlay')
repoPSt     =   path.join(db_root, 'PlayerStats/player')
repoModel   =   path.join(repoCode, 'ReinforcementLearning/NHL/playerstats/offVSdef/Automatic_classification/MODEL_backup_trainedonallseasons_rankstatprediction')
repoModel   =   path.join(repoCode, 'ReinforcementLearning/NHL/playerstats/offVSdef/Automatic_classification/MODEL_perceptron_1layer_10units_relu')
repoSave    =   None #path.join(repoCode, 'ReinforcementLearning/NHL/playbyplay/data')

In [4]:
repoPbP

'/Users/younes_zerouali/Documents/Stradigi/Databases/Hockey/PlayByPlay'

In [5]:
# ==== Before starting, recompute the model
# This should avoid path issues when reloading it
normalizer, pca, dtCols, CLS    =   do_ANN_training(repoPSt, repoPbP, repoCode, repoModel, minGames=0.2)

KeyboardInterrupt: 

In [5]:
# ==== First the game data

# Montreal received Ottawa on march 13, 2013, let's convert game date to game code
gameInfo    =   pickle.load( open(path.join(db_root, 'gamesInfo.p'), 'rb') )
gameInfo    =   gameInfo[gameInfo['gameDate']=='2013-03-13'][gameInfo['teamAbbrev']=='MTL']
gameId      =   gameInfo['gameId']
gameId      =   int( gameId.values.astype('str')[0][5:] )

# Now lets get game data 
season      =   '20122013'
mtlott      =   Game(repoPbP, repoPSt, season, gameId=gameId )

/Users/younes_zerouali/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [6]:
repoModel

'/Users/younes_zerouali/Documents/Stradigi/Code/NHL_stats_SL/ReinforcementLearning/NHL/playerstats/offVSdef/Automatic_classification/MODEL_perceptron_1layer_10units_relu'

In [7]:
# ==== Second the players data

# Need to load the data pre-processing variables
preprocessing  =   pickle.load(open(path.join(repoModel, 'baseVariables.p'), 'rb'))

# Need to load the classification model (for players' predicted ranking on trophies voting lists)
classifier          =   {'sess':tf.Session(), 'annX':[], 'annY':[]}
saver               =   tf.train.import_meta_graph(path.join(repoModel, path.basename(repoModel) + '.meta'))
graph               =   classifier['sess'].graph
classifier['annX']  =   graph.get_tensor_by_name('Input_to_the_network-player_features:0')
classifier['annY']  =   graph.get_tensor_by_name('prediction:0')
saver.restore(classifier['sess'], tf.train.latest_checkpoint(path.join(repoModel, './')))

# Pick players stats - last 30 games 
mtlott.pull_line_shifts(team='both', minduration=20)
mtlott.pick_regulartime()
mtlott.pick_equalstrength()
mtlott.pull_players_classes(preprocessing, classifier, nGames=30) #Here set number of n-back games to 30



NotFoundError: /home/younesz/Documents/Code/Python/ReinforcementLearning/NHL/playerstats/offVSdef/Automatic_classification/MODEL_perceptron_1layer_10units_relu

In [12]:
# Visualize all player's classes: 0=def, 1=off, 2=neutral
mtlott.player_classes


AttributeError: 'Game' object has no attribute 'player_classes'

In [13]:
# === Now we get the indices in the Q-values tables corresponding to lines

# Line translation table
linedict  = HockeySS(repoPbP, repoPSt)
linedict.make_line_dictionary()
linedict  = linedict.line_dictionary

# Get lines and translate them 
playersCode  =   mtlott.encode_line_players()
linesCode    =   np.array( [[mtlott.recode_line(linedict, a) for a in b] for b in lCode] )


KeyError: 'playersID'

In [22]:
# Visualize the line codes for each shift (first digit is the code of the away team's line - state-space, 
# second digit is the code of the home team's line - action space)
lComp

array([[ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [-1,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6, -1],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [-1,  6],
       [ 6, -1],
       [ 6,  6],
       [-1,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6],
       [ 6,  6]])

In [68]:
# ==== Now we want to evaluate the value of the first shift for the (home team)

# Load the Qvalues table
Qvalues = pickle.load( open(path.join(repoCode, 'ReinforcementLearning/NHL/playbyplay/data/stable/RL_action_values.p'), 'rb') )['action_values']

# Visualize it dimensions (period x differential x away line's code x home line's code)
print('Q-table dimensions: ', Qvalues.shape)

# Get the Q-value for that specific line
iShift = 0 # First shift
plList = list(mtlott.player_classes.loc[mtlott.lineShifts['playersID'].iloc[iShift][0]]['firstlast'].values) + list(mtlott.player_classes.loc[mtlott.lineShifts['playersID'].iloc[iShift][1]]['firstlast'].values)
diff   = mtlott.recode_differential( mtlott.lineShifts.iloc[iShift].differential )
period = mtlott.recode_period( mtlott.lineShifts.iloc[iShift].period )
print('First shift: \n\thome team: %s, %s, %s \n\taway team: %s, %s, %s \n\tQvalue: %.2f' %(plList[0], plList[1], plList[2], plList[3], plList[4], plList[5], Qvalues[period,diff,lComp[iShift,0], lComp[iShift,1]]) )
                                                                          


Q-table dimensions:  (3, 5, 10, 10)
First shift: 
	home team: DAVID DESHARNAIS, BRENDAN GALLAGHER, MAX PACIORETTY 
	away team: KYLE TURRIS, JAKOB SILFVERBERG, COLIN GREENING 
	Qvalue: 3.75
